In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

/var/folders/tk/zhnlhwsd0w734ckw7_z84vb80000gn/T/ipykernel_3090/4242059896.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(['EIN','NAME'],1)


APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1

In [3]:

# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:

# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 300].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [11]:

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  \
0                     0.0                   1.0                   0.0   
1                     0.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     0.0                   0.0                   0.0   
4                     0.0                   0.0                   0.0   

   APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  \
0                  0.0                  0.0                  0.0   
1                  1.0                  0.0                  0.0   
2                  0.0                  0.0                  1.0   
3                  1.0                  0.0                  0.0   
4                  1.0                  0.0                  0.0   

   APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  APPLICATION_TYPE_T8  \
0                  0.0                  0.0                  0.0   
1                  0.0                  0.0                  0.0   
2                  0.0                  0.0                  0.0   
3                  0.0                  0.0                  0.0   
4                  0.0                  0.0                  0.0   

   AFFILIATION_CompanySponsored  ...  ORGANIZATION_Trust  INCOME_AMT_0  \
0                           0.0  ...                 0.0           1.0   
1                           0.0  ...                 0.0           0.0   
2                           1.0  ...                 0.0           1.0   
3                           1.0  ...                 1.0           0.0   
4                           0.0  ...                 1.0           0.0   

   INCOME_AMT_1-9999  INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  \
0                0.0                     0.0                       0.0   
1                1.0                     0.0                       0.0   
2                0.0                     0.0                       0.0   
3                0.0                     1.0                       0.0   
4                0.0                     0.0                       1.0   

   INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  INCOME_AMT_Other  \
0               0.0                     0.0               0.0   
1               0.0                     0.0               0.0   
2               0.0                     0.0               0.0   
3               0.0                     0.0               0.0   
4               0.0                     0.0               0.0   

   SPECIAL_CONSIDERATIONS_N  SPECIAL_CONSIDERATIONS_Y  
0                       1.0                       0.0  
1                       1.0                       0.0  
2                       1.0                       0.0  
3                       1.0                       0.0  
4                       1.0                       0.0  

[5 rows x 39 columns]

In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

/var/folders/tk/zhnlhwsd0w734ckw7_z84vb80000gn/T/ipykernel_3090/108820884.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(application_cat,1)


STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                     0.0   
1       1   108590              1                     0.0   
2       1     5000              0                     0.0   
3       1     6692              1                     0.0   
4       1   142590              1                     0.0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                   1.0                   0.0                  0.0   
1                   0.0                   0.0                  1.0   
2                   0.0                   0.0                  0.0   
3                   0.0                   0.0                  1.0   
4                   0.0                   0.0                  1.0   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                  0.0                  0.0                  0.0  ...   
1                  0.0                  0.0                  0.0  ...   
2                  0.0                  1.0                  0.0  ...   
3                  0.0                  0.0                  0.0  ...   
4                  0.0                  0.0                  0.0  ...   

   ORGANIZATION_Trust  INCOME_AMT_0  INCOME_AMT_1-9999  \
0                 0.0           1.0                0.0   
1                 0.0           0.0                1.0   
2                 0.0           1.0                0.0   
3                 1.0           0.0                0.0   
4                 1.0           0.0                0.0   

   INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  INCOME_AMT_1M-5M  \
0                     0.0                       0.0               0.0   
1                     0.0                       0.0               0.0   
2                     0.0                       0.0               0.0   
3                     1.0                       0.0               0.0   
4                     0.0                       1.0               0.0   

   INCOME_AMT_25000-99999  INCOME_AMT_Other  SPECIAL_CONSIDERATIONS_N  \
0                     0.0               0.0                       1.0   
1                     0.0               0.0                       1.0   
2                     0.0               0.0                       1.0   
3                     0.0               0.0                       1.0   
4                     0.0               0.0                       1.0   

   SPECIAL_CONSIDERATIONS_Y  
0                       0.0  
1                       0.0  
2                       0.0  
3                       0.0  
4                       0.0  

[5 rows x 42 columns]

In [13]:

# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL',1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

/var/folders/tk/zhnlhwsd0w734ckw7_z84vb80000gn/T/ipykernel_3090/1945616859.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = application_df.drop('IS_SUCCESSFUL',1).values


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3360      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,821
Trainable params: 5,821
Non-trainable params: 0
_________________________________________________________________


2021-10-24 17:27:55.790732: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

2021-10-24 17:27:55.870254: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
804/804 [==============================] - 1s 663us/step - loss: 0.5688 - accuracy: 0.7234
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 0s 608us/step - loss: 0.5520 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 1s 671us/step - loss: 0.5497 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 1s 797us/step - loss: 0.5488 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 1s 830us/step - loss: 0.5474 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 1s 739us/step - loss: 0.5468 - accuracy: 0.7348
Epoch 8/100
804/804 [==============================] - 1s 624us/step - loss: 0.5460 - accuracy: 0.7355
Epoch 9/100
804/804 [==============================] - 1s 644us/step - loss: 0.5461 - accuracy: 0.7354
Epoch 10/100
804/804 [==============================] - 1s 661us/step - los

804/804 [==============================] - 1s 1ms/step - loss: 0.5354 - accuracy: 0.7416
Epoch 80/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5352 - accuracy: 0.7413: 0s - l
Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5354 - accuracy: 0.7415: 0s - loss: 0.5344 - accura
Epoch 82/100
804/804 [==============================] - 1s 974us/step - loss: 0.5355 - accuracy: 0.7417
Epoch 83/100
804/804 [==============================] - 1s 718us/step - loss: 0.5354 - accuracy: 0.7407
Epoch 84/100
804/804 [==============================] - 1s 688us/step - loss: 0.5350 - accuracy: 0.7406
Epoch 85/100
804/804 [==============================] - 1s 713us/step - loss: 0.5348 - accuracy: 0.7417
Epoch 86/100
804/804 [==============================] - 0s 576us/step - loss: 0.5351 - accuracy: 0.7417
Epoch 87/100
804/804 [==============================] - 1s 737us/step - loss: 0.5347 - accuracy: 0.7414
Epoch 88/100
804/804 [=========================

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5621 - accuracy: 0.7255
Loss: 0.5620794892311096, Accuracy: 0.7254810333251953


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization3/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization3/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 78/804 [=>............................] - ETA: 0s - loss: 0.5280 - accuracy: 0.7500  
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
149/804 [====>.........................] - ETA: 0s - loss: 0.5395 - accuracy: 0.7418
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
225/804 [=======>......................] - ETA: 0s - loss: 0.5417 - accuracy: 0.7365
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
359/804 [============>.................] - ETA: 0s - loss: 0.5349 - accuracy: 0.7411
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
480/804 [================>.............] - ETA: 0s - loss: 0.5357 - accuracy: 0.7395
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
538/804 [===================>..........] - ETA: 0s - loss: 0.5364 - accuracy: 0.7395
Epoch 00001: saving model to checkpoints/Optimization3/weights.01.hdf5
691/804 [========================>.....] - ETA: 0s

 68/804 [=>............................] - ETA: 0s - loss: 0.5472 - accuracy: 0.7307
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
128/804 [===>..........................] - ETA: 0s - loss: 0.5380 - accuracy: 0.7371
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
199/804 [======>.......................] - ETA: 0s - loss: 0.5373 - accuracy: 0.7378
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
357/804 [============>.................] - ETA: 0s - loss: 0.5378 - accuracy: 0.7372
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
434/804 [===============>..............] - ETA: 0s - loss: 0.5364 - accuracy: 0.7400
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
509/804 [=================>............] - ETA: 0s - loss: 0.5356 - accuracy: 0.7402
Epoch 00007: saving model to checkpoints/Optimization3/weights.07.hdf5
671/804 [========================>.....] - ETA: 0s - loss: 0.534

  1/804 [..............................] - ETA: 0s - loss: 0.6276 - accuracy: 0.5938
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
141/804 [====>.........................] - ETA: 0s - loss: 0.5280 - accuracy: 0.7460
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
216/804 [=======>......................] - ETA: 0s - loss: 0.5259 - accuracy: 0.7483
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
287/804 [=========>....................] - ETA: 0s - loss: 0.5305 - accuracy: 0.7442
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
445/804 [===============>..............] - ETA: 0s - loss: 0.5355 - accuracy: 0.7397
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
519/804 [==================>...........] - ETA: 0s - loss: 0.5346 - accuracy: 0.7402
Epoch 00013: saving model to checkpoints/Optimization3/weights.13.hdf5
597/804 [=====================>........] - ETA: 0s - loss: 0.535

  1/804 [..............................] - ETA: 0s - loss: 0.4328 - accuracy: 0.8125
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
 73/804 [=>............................] - ETA: 0s - loss: 0.5304 - accuracy: 0.7474
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
214/804 [======>.......................] - ETA: 0s - loss: 0.5358 - accuracy: 0.7414
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
282/804 [=========>....................] - ETA: 0s - loss: 0.5331 - accuracy: 0.7443
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
395/804 [=============>................] - ETA: 0s - loss: 0.5326 - accuracy: 0.7456
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
499/804 [=================>............] - ETA: 0s - loss: 0.5317 - accuracy: 0.7456
Epoch 00019: saving model to checkpoints/Optimization3/weights.19.hdf5
603/804 [=====================>........] - ETA: 0s - loss: 0.533

  1/804 [..............................] - ETA: 0s - loss: 0.4020 - accuracy: 0.8438
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
 77/804 [=>............................] - ETA: 0s - loss: 0.5392 - accuracy: 0.7277
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
152/804 [====>.........................] - ETA: 0s - loss: 0.5385 - accuracy: 0.7358
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
224/804 [=======>......................] - ETA: 0s - loss: 0.5383 - accuracy: 0.7377
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
392/804 [=============>................] - ETA: 0s - loss: 0.5359 - accuracy: 0.7390
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
466/804 [================>.............] - ETA: 0s - loss: 0.5325 - accuracy: 0.7412
Epoch 00025: saving model to checkpoints/Optimization3/weights.25.hdf5
543/804 [===================>..........] - ETA: 0s - loss: 0.533

...

  1/804 [..............................] - ETA: 0s - loss: 0.4994 - accuracy: 0.7500
Epoch 00073: saving model to checkpoints/Optimization3/weights.73.hdf5
 79/804 [=>............................] - ETA: 0s - loss: 0.5326 - accuracy: 0.7413
Epoch 00073: saving model to checkpoints/Optimization3/weights.73.hdf5
153/804 [====>.........................] - ETA: 0s - loss: 0.5287 - accuracy: 0.7471
Epoch 00073: saving model to checkpoints/Optimization3/weights.73.hdf5
306/804 [==========>...................] - ETA: 0s - loss: 0.5323 - accuracy: 0.7419
Epoch 00073: saving model to checkpoints/Optimization3/weights.73.hdf5
376/804 [=============>................] - ETA: 0s - loss: 0.5323 - accuracy: 0.7415
Epoch 00073: saving model to checkpoints/Optimization3/weights.73.hdf5
455/804 [===============>..............] - ETA: 0s - loss: 0.5328 - accuracy: 0.7411
Epoch 00073: saving model to checkpoints/Optimization3/weights.73.hdf5
532/804 [==================>...........] - ETA: 0s - loss: 0.534

Epoch 79/100
 84/804 [==>...........................] - ETA: 0s - loss: 0.5179 - accuracy: 0.7478
Epoch 00079: saving model to checkpoints/Optimization3/weights.79.hdf5
162/804 [=====>........................] - ETA: 0s - loss: 0.5221 - accuracy: 0.7440
Epoch 00079: saving model to checkpoints/Optimization3/weights.79.hdf5
238/804 [=======>......................] - ETA: 0s - loss: 0.5254 - accuracy: 0.7434
Epoch 00079: saving model to checkpoints/Optimization3/weights.79.hdf5
315/804 [==========>...................] - ETA: 0s - loss: 0.5295 - accuracy: 0.7398
Epoch 00079: saving model to checkpoints/Optimization3/weights.79.hdf5
479/804 [================>.............] - ETA: 0s - loss: 0.5299 - accuracy: 0.7421
Epoch 00079: saving model to checkpoints/Optimization3/weights.79.hdf5
557/804 [===================>..........] - ETA: 0s - loss: 0.5300 - accuracy: 0.7429
Epoch 00079: saving model to checkpoints/Optimization3/weights.79.hdf5
635/804 [======================>.......] - ETA: 0s 

  1/804 [..............................] - ETA: 0s - loss: 0.5381 - accuracy: 0.7812
Epoch 00085: saving model to checkpoints/Optimization3/weights.85.hdf5
162/804 [=====>........................] - ETA: 0s - loss: 0.5290 - accuracy: 0.7425
Epoch 00085: saving model to checkpoints/Optimization3/weights.85.hdf5
234/804 [=======>......................] - ETA: 0s - loss: 0.5278 - accuracy: 0.7447
Epoch 00085: saving model to checkpoints/Optimization3/weights.85.hdf5
312/804 [==========>...................] - ETA: 0s - loss: 0.5299 - accuracy: 0.7429
Epoch 00085: saving model to checkpoints/Optimization3/weights.85.hdf5
390/804 [=============>................] - ETA: 0s - loss: 0.5295 - accuracy: 0.7442
Epoch 00085: saving model to checkpoints/Optimization3/weights.85.hdf5
539/804 [===================>..........] - ETA: 0s - loss: 0.5301 - accuracy: 0.7431
Epoch 00085: saving model to checkpoints/Optimization3/weights.85.hdf5
614/804 [=====================>........] - ETA: 0s - loss: 0.528

  1/804 [..............................] - ETA: 0s - loss: 0.5287 - accuracy: 0.7500
Epoch 00091: saving model to checkpoints/Optimization3/weights.91.hdf5
 78/804 [=>............................] - ETA: 0s - loss: 0.5189 - accuracy: 0.7468
Epoch 00091: saving model to checkpoints/Optimization3/weights.91.hdf5
229/804 [=======>......................] - ETA: 0s - loss: 0.5290 - accuracy: 0.7421
Epoch 00091: saving model to checkpoints/Optimization3/weights.91.hdf5
306/804 [==========>...................] - ETA: 0s - loss: 0.5332 - accuracy: 0.7392
Epoch 00091: saving model to checkpoints/Optimization3/weights.91.hdf5
376/804 [=============>................] - ETA: 0s - loss: 0.5339 - accuracy: 0.7399
Epoch 00091: saving model to checkpoints/Optimization3/weights.91.hdf5
454/804 [===============>..............] - ETA: 0s - loss: 0.5343 - accuracy: 0.7399
Epoch 00091: saving model to checkpoints/Optimization3/weights.91.hdf5
628/804 [======================>.......] - ETA: 0s - loss: 0.533

  1/804 [..............................] - ETA: 0s - loss: 0.5897 - accuracy: 0.6875
Epoch 00097: saving model to checkpoints/Optimization3/weights.97.hdf5
 69/804 [=>............................] - ETA: 0s - loss: 0.5453 - accuracy: 0.7310
Epoch 00097: saving model to checkpoints/Optimization3/weights.97.hdf5
212/804 [======>.......................] - ETA: 0s - loss: 0.5297 - accuracy: 0.7401
Epoch 00097: saving model to checkpoints/Optimization3/weights.97.hdf5
282/804 [=========>....................] - ETA: 0s - loss: 0.5291 - accuracy: 0.7436
Epoch 00097: saving model to checkpoints/Optimization3/weights.97.hdf5
350/804 [============>.................] - ETA: 0s - loss: 0.5302 - accuracy: 0.7435
Epoch 00097: saving model to checkpoints/Optimization3/weights.97.hdf5
499/804 [=================>............] - ETA: 0s - loss: 0.5293 - accuracy: 0.7464
Epoch 00097: saving model to checkpoints/Optimization3/weights.97.hdf5
616/804 [=====================>........] - ETA: 0s - loss: 0.530

In [20]:

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5714 - accuracy: 0.7262
Loss: 0.5713513493537903, Accuracy: 0.7261807322502136


In [21]:

# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization3.h5')